In [58]:
import pandas as pd

In [4]:
feature_selection = pd.read_csv("train_test_data/feature selection rationalization.csv")
mri = pd.read_csv("train_test_data/all_MRI_features.csv")

In [78]:
regions = {
    'caudalanteriorcingulate',
    'caudalmiddlefrontal',
    'cuneus',
    'entorhinal',
    'fusiform',
    'inferiorparietal',
    'inferiortemporal',
    'insula',
    'isthmuscingulate',
    'lateraloccipital',
    'lateralorbitofrontal',
    'lingual',
    'medialorbitofrontal',
    'middletemporal',
    'paracentral',
    'parahippocampal',
    'parsopercularis',
    'parsorbitalis',
    'parstriangularis',
    'pericalcarine',
    'postcentral',
    'posteriorcingulate',
    'precentral',
    'precuneus',
    'rostralanteriorcingulate',
    'rostralmiddlefrontal',
    'superiorfrontal',
    'superiorparietal',
    'superiortemporal',
    'supramarginal',
    'transversetemporal'
}

In [19]:
print({c for c in mri.columns})

{'Zernike moments: component 19_left_cortical_surface_label_shapes_ctx-rh-middletemporal_ID', 'Zernike moments: component 21_left_cortical_surface_label_shapes_ctx-lh-parahippocampal_ID', 'Zernike moments: component 35_right_cortical_surface_sulcus_shapes_paracentral sulcus_ID', 'Zernike moments: component 14_left_cortical_surface_label_shapes_ctx-rh-transversetemporal_ID', 'mean position in standard space: z_left_cortical_surface_sulcus_shapes_intraparietal sulcus_ID', 'freesurfer curvature: MAD_right_cortical_surface_sulcus_shapes_postcentral sulcus_ID', 'freesurfer curvature: kurtosis_right_cortical_surface_label_shapes_ctx-rh-superiortemporal_ID', 'mean curvature: kurtosis_right_cortical_surface_label_shapes_ctx-rh-cuneus_ID', 'travel depth: median_right_cortical_surface_label_shapes_ctx-rh-parahippocampal_ID', 'geodesic depth: 25%_left_cortical_surface_label_shapes_ctx-lh-postcentral_ID', 'Laplace-Beltrami spectrum: component 3_right_cortical_surface_label_shapes_ctx-rh-entorhinal

In [12]:
{thing.lower().replace("-", " ") for feature in list(mri.columns) for thing in feature.split("_")}

{"1st transverse temporal sulcus and heschl's sulcus",
 'adhd subtype',
 'anterior occipital sulcus',
 'ants',
 'area',
 'asd',
 'brain stem',
 'calcarine fissure',
 'central sulcus',
 'cingulate sulcus',
 'circular sulcus',
 'collateral sulcus',
 'cortex',
 'cortical',
 'csf',
 'ctx lh bankssts',
 'ctx lh caudalanteriorcingulate',
 'ctx lh caudalmiddlefrontal',
 'ctx lh cuneus',
 'ctx lh entorhinal',
 'ctx lh frontalpole',
 'ctx lh fusiform',
 'ctx lh inferiorparietal',
 'ctx lh inferiortemporal',
 'ctx lh insula',
 'ctx lh isthmuscingulate',
 'ctx lh lateraloccipital',
 'ctx lh lateralorbitofrontal',
 'ctx lh lingual',
 'ctx lh medialorbitofrontal',
 'ctx lh middletemporal',
 'ctx lh paracentral',
 'ctx lh parahippocampal',
 'ctx lh parsopercularis',
 'ctx lh parsorbitalis',
 'ctx lh parstriangularis',
 'ctx lh pericalcarine',
 'ctx lh postcentral',
 'ctx lh posteriorcingulate',
 'ctx lh precentral',
 'ctx lh precuneus',
 'ctx lh rostralanteriorcingulate',
 'ctx lh rostralmiddlefront

In [27]:
{thing.lower().replace("-", " ") for feature in list(mri.columns) for thing in feature.split("_")}

{"1st transverse temporal sulcus and heschl's sulcus",
 'adhd subtype',
 'anterior occipital sulcus',
 'ants',
 'area',
 'asd',
 'brain stem',
 'calcarine fissure',
 'central sulcus',
 'cingulate sulcus',
 'circular sulcus',
 'collateral sulcus',
 'cortex',
 'cortical',
 'csf',
 'ctx lh bankssts',
 'ctx lh caudalanteriorcingulate',
 'ctx lh caudalmiddlefrontal',
 'ctx lh cuneus',
 'ctx lh entorhinal',
 'ctx lh frontalpole',
 'ctx lh fusiform',
 'ctx lh inferiorparietal',
 'ctx lh inferiortemporal',
 'ctx lh insula',
 'ctx lh isthmuscingulate',
 'ctx lh lateraloccipital',
 'ctx lh lateralorbitofrontal',
 'ctx lh lingual',
 'ctx lh medialorbitofrontal',
 'ctx lh middletemporal',
 'ctx lh paracentral',
 'ctx lh parahippocampal',
 'ctx lh parsopercularis',
 'ctx lh parsorbitalis',
 'ctx lh parstriangularis',
 'ctx lh pericalcarine',
 'ctx lh postcentral',
 'ctx lh posteriorcingulate',
 'ctx lh precentral',
 'ctx lh precuneus',
 'ctx lh rostralanteriorcingulate',
 'ctx lh rostralmiddlefront

In [31]:
set(feature_selection.Region)

{'amygdala',
 'anterior cingulate cortex',
 'basal ganglia',
 'caudate',
 'cerebellum',
 'corpus callosum',
 'frontal lobes',
 'intracranial volume',
 'parietal cortex, left inferior',
 'parietal lobes',
 'prefrontal cortex',
 'prefrontal cortex, bilateral dorsomedial',
 'whole brain'}

In [38]:
{'amygdala', 'caudalanteriorcingulate', 'cerebral', 'cerebellum', 'caudate', 'parietal', 'frontal'}
{'area', 'travel-depth', 'freesurfer-thickness', 'mean-curvature', 'volume', 'median', 'mad', 'laplace-beltrami-spectrum', 'zernike-moments'}

{'area',
 'freesurfer-thickness',
 'laplace-beltrami-spectrum',
 'mad',
 'mean-curvature',
 'median',
 'travel-depth',
 'volume',
 'zernike-moments'}

In [37]:
{'vessel', 'lesion', 'ants', 'geodesic', 'freesurfer-depth'}

{'ants', 'freesurfer-depth', 'geodesic', 'lesion', 'vessel'}

In [96]:
"Acumbens", "Caudate", "Cerebral-White-Matter", "Hippocampus", "Pallidum", "Putamen", "Thalamus", "UnsegmentedWhiteMatter"

[2, 3, 4, 5, 6, 7, 8, 9, 10]

In [38]:
len({c for c in mri.columns if (
    "-rh" in c and "left" in c
)})

3100

In [37]:
len({c for c in mri.columns if (
    "lh" in c and "_right" in c
)})

3100

In [39]:
len({c for c in mri.columns if (
    "-rh" in c and "right" in c
)})

3100

In [36]:
len({c for c in mri.columns if (
    "lh" in c and "left" in c
)})

3100

In [41]:
mri[[c for c in mri.columns if (
    "-rh" in c and "eft" in c
) or (
    "lh" in c and "ight" in c
)]]

,Laplace-Beltrami spectrum: component 10_left_cortical_surface_label_shapes_ctx-rh-caudalanteriorcingulate_ID,Laplace-Beltrami spectrum: component 10_left_cortical_surface_label_shapes_ctx-rh-caudalmiddlefrontal_ID,Laplace-Beltrami spectrum: component 10_left_cortical_surface_label_shapes_ctx-rh-cuneus_ID,Laplace-Beltrami spectrum: component 10_left_cortical_surface_label_shapes_ctx-rh-entorhinal_ID,Laplace-Beltrami spectrum: component 10_left_cortical_surface_label_shapes_ctx-rh-fusiform_ID,Laplace-Beltrami spectrum: component 10_left_cortical_surface_label_shapes_ctx-rh-inferiorparietal_ID,Laplace-Beltrami spectrum: component 10_left_cortical_surface_label_shapes_ctx-rh-inferiortemporal_ID,Laplace-Beltrami spectrum: component 10_left_cortical_surface_label_shapes_ctx-rh-insula_ID,Laplace-Beltrami spectrum: component 10_left_cortical_surface_label_shapes_ctx-rh-isthmuscingulate_ID,Laplace-Beltrami spectrum: component 10_left_cortical_surface_label_shapes_ctx-rh-lateraloccipital_ID,...,travel depth: skew_right_cortical_surface_label_shapes_ctx-lh-posteriorcingulate_ID,travel depth: skew_right_cortical_surface_label_shapes_ctx-lh-precentral_ID,travel depth: skew_right_cortical_surface_label_shapes_ctx-lh-precuneus_ID,travel depth: skew_right_cortical_surface_label_shapes_ctx-lh-rostralanteriorcingulate_ID,travel depth: skew_right_cortical_surface_label_shapes_ctx-lh-rostralmiddlefrontal_ID,travel depth: skew_right_cortical_surface_label_shapes_ctx-lh-superiorfrontal_ID,travel depth: skew_right_cortical_surface_label_shapes_ctx-lh-superiorparietal_ID,travel depth: skew_right_cortical_surface_label_shapes_ctx-lh-superiortemporal_ID,travel depth: skew_right_cortical_surface_label_shapes_ctx-lh-supramarginal_ID,travel depth: skew_right_cortical_surface_label_shapes_ctx-lh-transversetemporal_ID
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# keep whole brain volume, but split out other sums

In [128]:
mri_subset = mri[["EID"]].copy()
mri_subset = mri_subset.assign(
    left_cortical_grey_matter_volume=mri[
        [c for c in mri.columns if "volume_per_freesurfer" in c and "ctx-lh" in c]
    ].sum(axis=1),
    right_cortical_grey_matter_volume=mri[
        [c for c in mri.columns if "volume_per_freesurfer" in c and "ctx-rh" in c]
    ].sum(axis=1),
    left_cortical_white_matter_volume=mri[
        [c for c in mri.columns if "volume_per_freesurfer" in c and "wm-lh" in c]
    ].sum(axis=1),
    right_cortical_white_matter_volume=mri[
        [c for c in mri.columns if "volume_per_freesurfer" in c and "wm-rh" in c]
    ].sum(axis=1),
    csf_volume=mri['volume_volume_per_freesurfer_label_CSF_ID'],
    whole_brain_volume=mri[
        [c for c in mri.columns if "volume_per_freesurfer" in c]
    ].sum(axis=1)
)
mri_subset = pd.concat(
    [
        mri_subset,
        pd.DataFrame(
            [pd.Series(
                mri[
                    [c for c in mri.columns if "{0}: {1}_{2}".format(
                        feature,
                        statistic,
                        hemisphere
                    ) in c and "ctx-{0}h-{1}".format(
                        hemisphere[0],
                        region
                    ) in c]
                ].sum(axis=1),
                name="{1}_{2}_{3}_{0}".format(
                    "-".join(
                        feature.split(" ")
                    ),
                    statistic,
                    hemisphere,
                    region
                )
            ) for feature in [
                "freesurfer thickness",
                "travel depth",
                "mean curvature"
            ] for statistic in [
                "median",
                "MAD"
            ] for hemisphere in [
                "left",
                "right"
            ] for region in regions]
        ).T,
        pd.DataFrame(
            [pd.Series(
                mri[
                    [c for c in mri.columns if "{0}_{1}".format(
                feature,
                hemisphere
                    ) in c and "ctx-{0}h".format(
                        hemisphere[0],
                        region
                    ) in c]
                ].sum(axis=1),
                name="{1}_{2}_{0}".format(
                    "-".join(
                        feature.split(" ")
                    ),
                    hemisphere,
                    region
                )
            ) for feature in [
                "area"
            ] for hemisphere in [
                "left",
                "right"
            ] for region in regions]
        ).T,
        pd.DataFrame(
            [pd.Series(
                mri[
                    [c for c in mri.columns if feature in c and region in c and hemisphere in c]
                ].sum(axis=1),
                name="{1}_{0}_{2}".format(
                    "-".join(
                        region.split("_")
                    ),
                    hemisphere,
                    "-".join(
                        feature.split("_")
                    )
                ).lower()
            ) for region in [
                "Amygdala",
                "Acumbens",
                "Caudate",
                "Cerebral-White-Matter",
                "Hippocampus",
                "Pallidum",
                "Putamen",
                "Thalamus",
                "UnsegmentedWhiteMatter"
            ] for hemisphere in [
                "Left",
                "Right"
            ] for feature in [
                "volume_per_freesurfer_label"
            ]]
        ).T
    ],
    axis=1
)

In [129]:
mri_subset

,EID,csf_volume,left_cortical_grey_matter_volume,left_cortical_white_matter_volume,right_cortical_grey_matter_volume,right_cortical_white_matter_volume,whole_brain_volume,median_left_lateraloccipital_freesurfer-thickness,median_left_caudalmiddlefrontal_freesurfer-thickness,median_left_insula_freesurfer-thickness,...,left_hippocampus_volume-per-freesurfer-label,right_hippocampus_volume-per-freesurfer-label,left_pallidum_volume-per-freesurfer-label,right_pallidum_volume-per-freesurfer-label,left_putamen_volume-per-freesurfer-label,right_putamen_volume-per-freesurfer-label,left_thalamus_volume-per-freesurfer-label,right_thalamus_volume-per-freesurfer-label,left_unsegmentedwhitematter_volume-per-freesurfer-label,right_unsegmentedwhitematter_volume-per-freesurfer-label
0,NDARAE012DGA,564.641,302144.483,189755.286,300732.627,188385.404,1107431.397,2.444076,2.995205,3.436579,...,3922.798,3882.868,2073.252,2434.663,4324.138,3805.569,6939.507,6871.423,20849.748,20443.289
1,NDARAM675UR8,976.844,278853.111,187160.847,274404.582,193038.290,1057275.386,2.279456,2.780890,3.326810,...,3800.886,3703.099,1821.599,1846.174,5313.765,5523.674,8171.597,7844.958,20254.153,21204.886
2,NDARAN814UPR,420.817,234745.958,151511.151,234590.330,154621.719,874746.224,2.021367,2.298171,3.247139,...,3367.563,3445.890,1537.878,1479.516,4818.410,5046.737,6232.397,6232.397,13974.516,14263.764
3,NDARAP176AD1,887.206,313159.791,194066.795,315291.034,194133.351,1151462.313,2.565473,3.109242,3.612399,...,3856.506,4166.747,2208.033,1947.963,4543.541,5397.471,8656.534,7580.418,23129.312,24815.157
4,NDARBF805EHN,1251.178,282270.064,212409.408,280858.138,212132.960,1148386.993,2.317927,2.917776,3.080252,...,4423.148,4672.462,1965.332,1766.700,4929.455,5247.369,8832.986,8307.225,34471.888,33639.988
5,NDARBM173BJG,841.075,297046.297,202731.260,306159.905,205231.448,1132725.453,2.582865,3.119935,3.409763,...,4371.234,4664.561,2161.813,2385.519,4385.567,4430.104,7509.779,7564.553,15844.251,16736.517
6,NDARBR740NKV,689.575,225881.525,163455.305,223613.142,161755.174,896562.258,2.665291,3.120793,3.306933,...,3379.274,3462.208,1921.287,1771.291,4144.615,4530.613,7289.936,6237.400,24659.334,24281.527
7,NDARBV680AA8,764.851,241086.658,167434.712,242170.963,165694.605,940366.463,2.324691,2.693396,3.402360,...,3974.766,4330.058,1681.750,1568.609,4229.204,4279.887,7145.773,7160.620,22639.886,23511.734
8,NDARBX121UM9,852.890,296150.319,202209.084,300062.554,205508.012,1148662.681,2.430216,3.124940,3.343808,...,4347.382,4422.637,2339.559,2159.357,5822.789,6168.348,9019.846,8014.398,26348.454,26985.818
9,NDARBX974XDR,982.448,269537.728,188734.876,271167.803,190395.155,1063424.439,2.262045,3.079203,3.297931,...,4198.056,4131.502,2127.698,1904.996,4129.454,4617.862,8608.575,7303.594,28653.781,29038.261


In [130]:
[c for feature in [
    "freesurfer thickness",
    "travel depth",
    "mean curvature"
] for statistic in [
    "median",
    "MAD"
] for hemisphere in [
    "left",
    "right"
] for c in mri.columns if "{0}: {1}_{2}".format(
        feature,
        statistic,
        hemisphere
    ) in c and "ctx" in c]

['freesurfer thickness: median_left_cortical_surface_label_shapes_ctx-lh-caudalanteriorcingulate_ID',
 'freesurfer thickness: median_left_cortical_surface_label_shapes_ctx-lh-caudalmiddlefrontal_ID',
 'freesurfer thickness: median_left_cortical_surface_label_shapes_ctx-lh-cuneus_ID',
 'freesurfer thickness: median_left_cortical_surface_label_shapes_ctx-lh-entorhinal_ID',
 'freesurfer thickness: median_left_cortical_surface_label_shapes_ctx-lh-fusiform_ID',
 'freesurfer thickness: median_left_cortical_surface_label_shapes_ctx-lh-inferiorparietal_ID',
 'freesurfer thickness: median_left_cortical_surface_label_shapes_ctx-lh-inferiortemporal_ID',
 'freesurfer thickness: median_left_cortical_surface_label_shapes_ctx-lh-insula_ID',
 'freesurfer thickness: median_left_cortical_surface_label_shapes_ctx-lh-isthmuscingulate_ID',
 'freesurfer thickness: median_left_cortical_surface_label_shapes_ctx-lh-lateraloccipital_ID',
 'freesurfer thickness: median_left_cortical_surface_label_shapes_ctx-lh-l

In [131]:
len({
    c for component in range(
        2,
        11,
        1
    ) for hemisphere in [
        "left",
        "right"
    ] for region in {
        'caudalanteriorcingulate',
        'caudalmiddlefrontal',
        'cuneus',
        'entorhinal',
        'fusiform',
        'inferiorparietal',
        'inferiortemporal',
        'insula',
        'isthmuscingulate',
        'lateraloccipital',
        'lateralorbitofrontal',
        'lingual',
        'medialorbitofrontal',
        'middletemporal',
        'paracentral',
        'parahippocampal',
        'parsopercularis',
        'parsorbitalis',
        'parstriangularis',
        'pericalcarine',
        'postcentral',
        'posteriorcingulate',
        'precentral',
        'precuneus',
        'rostralanteriorcingulate',
        'rostralmiddlefrontal',
        'superiorfrontal',
        'superiorparietal',
        'superiortemporal',
        'supramarginal',
        'transversetemporal'
    } for c in mri.columns if "Laplace-Beltrami spectrum: "
    "component {0}_{1}".format(
        str(component),
        hemisphere
    ) in c and "ctx" in c and "h-{0}".format(
        region
    ) in c
})/2

558.0

In [132]:
only_one_value = {c for c in mri_subset.columns if mri_subset[c].nunique()==1}
print(only_one_value)

{'left_acumbens_volume-per-freesurfer-label', 'right_acumbens_volume-per-freesurfer-label'}


In [133]:
mri_subset.drop(
    only_one_value,
    axis=1,
    inplace=True
)

In [134]:
mri_subset.to_csv("train_test_data/mri_subset.csv", index=False)

In [135]:
qdf = pd.read_csv("train_test_data/drop_aggregates.csv")

In [138]:
qdf = qdf.merge(
    mri_subset,
    on="EID",
    how="outer"
)
qdf.to_csv(
    "train_test_data/questions_and_mri_regions.csv",
    index=False
)

In [139]:
qdf

,APQ_P_01,APQ_P_02,APQ_P_03,APQ_P_04,APQ_P_05,APQ_P_06,APQ_P_07,APQ_P_08,APQ_P_09,APQ_P_10,...,left_hippocampus_volume-per-freesurfer-label,right_hippocampus_volume-per-freesurfer-label,left_pallidum_volume-per-freesurfer-label,right_pallidum_volume-per-freesurfer-label,left_putamen_volume-per-freesurfer-label,right_putamen_volume-per-freesurfer-label,left_thalamus_volume-per-freesurfer-label,right_thalamus_volume-per-freesurfer-label,left_unsegmentedwhitematter_volume-per-freesurfer-label,right_unsegmentedwhitematter_volume-per-freesurfer-label
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.0,5.0,2.0,4.0,4.0,NaN,4.0,2.0,5.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,5.0,2.0,3.0,5.0,2.0,4.0,1.0,5.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5.0,5.0,3.0,2.0,3.0,3.0,3.0,2.0,5.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,5.0,3.0,5.0,5.0,1.0,5.0,3.0,5.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,3.0,4.0,3.0,2.0,3.0,3.0,5.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5.0,5.0,2.0,5.0,3.0,1.0,4.0,1.0,5.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4.0,4.0,3.0,3.0,2.0,2.0,4.0,2.0,5.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,5.0,5.0,3.0,4.0,3.0,1.0,4.0,3.0,5.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4.0,4.0,3.0,3.0,4.0,1.0,3.0,2.0,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
